In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os
import json
import math
import time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
# Load API Credentials
with open('/Users/casta/.secret/yelp_api.json') as f:  
    login = json.load(f)

# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

# Set API call parameters
LOCATION = 'San Francisco, CA'
TERM = 'sushi'

# Specify JSON_FILE filename (can include a folder)
JSON_FILE = "Data/results_in_progress_SF_sushi.json"

# Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)


In [3]:
# If it does not exist, create an empty file
if not file_exists:
    # Create any needed folders
    folder = os.path.dirname(JSON_FILE)
    if len(folder) > 0:
        os.makedirs(folder, exist_ok=True)
    
    # Inform user and save empty list
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    with open(JSON_FILE, 'w') as f:
        json.dump([], f)
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_SF_sushi.json not found. Saving empty list to file.


In [4]:
# Load previous results and use len of results for offset
with open(JSON_FILE, 'r') as f:
    previous_results = json.load(f)
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [5]:
# Perform API calls and retrieve results
results = yelp_api.search_query(location=LOCATION, term=TERM, offset=n_results)

In [6]:
total_results = results['total']
total_results

1400

In [7]:
results_per_page = len(results['businesses'])
results_per_page

20

In [8]:
n_pages = math.ceil((total_results - n_results) / results_per_page)
n_pages

70

In [9]:
# Loop through remaining pages to extract all results
for i in tqdm_notebook(range(1,11)):
    # Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 API calls. Stopping loop.')
        break
    
    # Perform API call with offset
    results = yelp_api.search_query(location=LOCATION, term=TERM, offset=n_results)
    
    # Append new results and save to file
    previous_results.extend(results['businesses'])
    with open(JSON_FILE, 'w') as f:
        json.dump(previous_results, f)
    
    time.sleep(0.2)

  0%|          | 0/10 [00:00<?, ?it/s]

In [11]:
# Load final results
final_df = pd.read_json(JSON_FILE)
final_df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,KutHWQT6Ha5ZRplvZ5mLXg,toyama-sushi-san-francisco-2,Toyama Sushi,https://s3-media1.fl.yelpcdn.com/bphoto/mD6OR_...,False,https://www.yelp.com/biz/toyama-sushi-san-fran...,432,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.5,"{'latitude': 37.780943, 'longitude': -122.4594...","[pickup, delivery]",$$,"{'address1': '3727 Geary Blvd', 'address2': ''...",+14152219811,(415) 221-9811,3009.190785
1,cL0q9S4bqwpbAN9ZKh-Zeg,nara-restaurant-and-sake-bar-san-francisco,Nara Restaurant & Sake Bar,https://s3-media1.fl.yelpcdn.com/bphoto/7zzlpJ...,False,https://www.yelp.com/biz/nara-restaurant-and-s...,1942,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.5,"{'latitude': 37.7721665317965, 'longitude': -1...","[restaurant_reservation, pickup, delivery]",$$,"{'address1': '518 Haight St', 'address2': '', ...",+14156386124,(415) 638-6124,1351.664131
2,ZGAjGYDFBJV1Z0JO2fanKA,rocket-sushi-san-francisco-2,Rocket Sushi,https://s3-media3.fl.yelpcdn.com/bphoto/rpDSk5...,False,https://www.yelp.com/biz/rocket-sushi-san-fran...,231,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.5,"{'latitude': 37.77255, 'longitude': -122.42193}","[pickup, delivery]",$$,"{'address1': '1695 Market St', 'address2': Non...",+14158729368,(415) 872-9368,1816.809612
3,D1K_lFglA5fOYquqltYcdg,ebisu-san-francisco,Ebisu,https://s3-media1.fl.yelpcdn.com/bphoto/0wxdWf...,False,https://www.yelp.com/biz/ebisu-san-francisco?a...,2002,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.0,"{'latitude': 37.7644547169329, 'longitude': -1...",[delivery],$$,"{'address1': '1283 9th Ave', 'address2': '', '...",+14155661770,(415) 566-1770,2664.013118
4,6TL8udi-vy_5rH-wikMSoA,saru-sushi-bar-san-francisco-7,Saru Sushi Bar,https://s3-media1.fl.yelpcdn.com/bphoto/5-ugy0...,False,https://www.yelp.com/biz/saru-sushi-bar-san-fr...,970,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.5,"{'latitude': 37.751706, 'longitude': -122.4288...",[delivery],$$$,"{'address1': '3856 24th St', 'address2': '', '...",+14154004510,(415) 400-4510,1222.144191


In [12]:
final_df.tail()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
195,RxIFKVvc8iVBPIVpe55Byg,bon-nene-san-francisco,"Bon, Nene",https://s3-media1.fl.yelpcdn.com/bphoto/f204PD...,False,https://www.yelp.com/biz/bon-nene-san-francisc...,357,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.0,"{'latitude': 37.75762490446, 'longitude': -122...","[pickup, delivery]",$$,"{'address1': '2850 21st St', 'address2': '', '...",+14158729332,(415) 872-9332,2211.883792
196,rUQmWnDbZWo0AN9j1i3cAQ,woori-food-market-san-francisco-6,Woori Food Market,https://s3-media2.fl.yelpcdn.com/bphoto/yzIlrh...,False,https://www.yelp.com/biz/woori-food-market-san...,144,"[{'alias': 'catering', 'title': 'Caterers'}, {...",4.5,"{'latitude': 37.78375, 'longitude': -122.43249}","[pickup, delivery]",$,"{'address1': '1528 Fillmore St', 'address2': '...",+14156739887,(415) 673-9887,2559.443481
197,jefsqzA1dIlsszG9Y6xwMQ,hina-yakitori-san-francisco-2,Hina Yakitori,https://s3-media1.fl.yelpcdn.com/bphoto/l19sWB...,False,https://www.yelp.com/biz/hina-yakitori-san-fra...,105,"[{'alias': 'japanese', 'title': 'Japanese'}]",4.5,"{'latitude': 37.77707, 'longitude': -122.43792}","[pickup, delivery]",$$$$,"{'address1': '808 Divisadero St', 'address2': ...",+14158171944,(415) 817-1944,1809.958848
198,5BsM7F6uetq0kYEXZ4TImQ,sushi-delight-san-francisco-2,Sushi Delight,https://s3-media1.fl.yelpcdn.com/bphoto/3rRoRI...,False,https://www.yelp.com/biz/sushi-delight-san-fra...,369,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",3.0,"{'latitude': 37.77032, 'longitude': -122.42583}","[pickup, delivery]",$$,"{'address1': '1946 Market St', 'address2': Non...",+14156213622,(415) 621-3622,1391.713721
199,LysKaLPCFOdsUnlJC48Trg,gozu-san-francisco,Gozu,https://s3-media2.fl.yelpcdn.com/bphoto/3Loftz...,False,https://www.yelp.com/biz/gozu-san-francisco?ad...,107,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.5,"{'latitude': 37.791243, 'longitude': -122.39198}",[delivery],$$$$,"{'address1': '201 Spear St', 'address2': 'Ste ...",+14155239745,(415) 523-9745,5162.267240
